In [1]:
import csv, os, requests, time, gmaps, pandas as pd, matplotlib.pyplot as plt, numpy as np
from config import *
gmaps.configure(api_key=gkey)

In [2]:
yelpapi_csv = os.path.join("yelpapidata.csv")
yelp_raw = pd.read_csv(yelpapi_csv)
yelp_raw.head()

,search_city,search_cuisine,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph
0,"Montgomery, Alabama",American,BhPj9fLZph1TcofvuTZT1w,Urban Cookhouse,https://s3-media1.fl.yelpcdn.com/bphoto/XY8xxa...,https://www.yelp.com/biz/urban-cookhouse-montg...,False,142,4.5,$,32.360651,-86.157520,7712 Eastchase Pkwy,Montgomery,AL,36117,US,(334) 239-7801
1,"Montgomery, Alabama",American,4uVYV49MDmm3tunRg6hnew,Stockyard Grill,https://s3-media2.fl.yelpcdn.com/bphoto/gfK2DK...,https://www.yelp.com/biz/stockyard-grill-montg...,False,51,4.5,$,32.317361,-86.350667,4500 Mobile Hwy,Montgomery,AL,36108,US,(334) 288-2575
2,"Montgomery, Alabama",American,s2uIZz572lvzeeaT40xm5g,Plantation House and Garden,https://s3-media3.fl.yelpcdn.com/bphoto/E4UqZ2...,https://www.yelp.com/biz/plantation-house-and-...,False,14,4.0,$$,32.479502,-86.369687,3240 Grandview Rd,Millbrook,AL,36054,US,(334) 285-1466
3,"Montgomery, Alabama",American,TmPADgXsze5_gfTZPZ24vQ,Southern Comfort,https://s3-media2.fl.yelpcdn.com/bphoto/3WRpNu...,https://www.yelp.com/biz/southern-comfort-hope...,False,193,4.5,$$,32.269824,-86.363997,210 Wasden Rd,Hope Hull,AL,36043,US,(334) 280-0012
4,"Montgomery, Alabama",American,u-pZS8ZvEm8QwJIMrf1Wbw,Common Bond Brewers,https://s3-media3.fl.yelpcdn.com/bphoto/xNl0HP...,https://www.yelp.com/biz/common-bond-brewers-m...,False,16,5.0,$,32.377410,-86.313590,424 Bibb St,Montgomery,AL,36104,US,(334) 676-2287


In [3]:
yelp_raw.describe()

,review_count,rating,lat,long
count,27851.000000,27851.000000,27850.000000,27850.000000
mean,177.127679,3.967398,38.425217,-92.894844
std,292.762941,0.604082,5.160674,18.784798
min,1.000000,1.000000,21.262370,-158.105524
25%,30.000000,3.500000,35.493787,-104.917115
50%,88.000000,4.000000,39.667450,-89.220339
75%,212.000000,4.500000,41.822154,-78.533204
max,9460.000000,5.000000,58.398782,157.941630


In [4]:
yelp_format_rd1 = yelp_raw.copy()
#yelp_format_rd1["review_count"] = yelp_format_rd1.loc[:, "review_count"].astype(float)
#yelp_format_rd1["rating"] = yelp_format_rd1.loc[:, "rating"].astype(float)
yelp_format_rd1['price_attribute'] = yelp_format_rd1['price'].replace(
    {'$': '1.Affordable', '$$': '2.Moderate', '$$$': '3.Expensive'})
yelp_format_rd1.head()

,search_city,search_cuisine,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph,price_attribute
0,"Montgomery, Alabama",American,BhPj9fLZph1TcofvuTZT1w,Urban Cookhouse,https://s3-media1.fl.yelpcdn.com/bphoto/XY8xxa...,https://www.yelp.com/biz/urban-cookhouse-montg...,False,142,4.5,$,32.360651,-86.157520,7712 Eastchase Pkwy,Montgomery,AL,36117,US,(334) 239-7801,1.Affordable
1,"Montgomery, Alabama",American,4uVYV49MDmm3tunRg6hnew,Stockyard Grill,https://s3-media2.fl.yelpcdn.com/bphoto/gfK2DK...,https://www.yelp.com/biz/stockyard-grill-montg...,False,51,4.5,$,32.317361,-86.350667,4500 Mobile Hwy,Montgomery,AL,36108,US,(334) 288-2575,1.Affordable
2,"Montgomery, Alabama",American,s2uIZz572lvzeeaT40xm5g,Plantation House and Garden,https://s3-media3.fl.yelpcdn.com/bphoto/E4UqZ2...,https://www.yelp.com/biz/plantation-house-and-...,False,14,4.0,$$,32.479502,-86.369687,3240 Grandview Rd,Millbrook,AL,36054,US,(334) 285-1466,2.Moderate
3,"Montgomery, Alabama",American,TmPADgXsze5_gfTZPZ24vQ,Southern Comfort,https://s3-media2.fl.yelpcdn.com/bphoto/3WRpNu...,https://www.yelp.com/biz/southern-comfort-hope...,False,193,4.5,$$,32.269824,-86.363997,210 Wasden Rd,Hope Hull,AL,36043,US,(334) 280-0012,2.Moderate
4,"Montgomery, Alabama",American,u-pZS8ZvEm8QwJIMrf1Wbw,Common Bond Brewers,https://s3-media3.fl.yelpcdn.com/bphoto/xNl0HP...,https://www.yelp.com/biz/common-bond-brewers-m...,False,16,5.0,$,32.377410,-86.313590,424 Bibb St,Montgomery,AL,36104,US,(334) 676-2287,1.Affordable


In [5]:
yelp_format_rd1.columns

Index(['search_city', 'search_cuisine', 'y_id', 'name', 'image_url', 'url',
       'closed', 'review_count', 'rating', 'price', 'lat', 'long', 'add1',
       'city', 'state', 'zip_code', 'country', 'disp_ph', 'price_attribute'],
      dtype='object')

In [6]:
yelp_format_rd1.count()

search_city        27851
search_cuisine     27851
y_id               27851
name               27851
image_url          27532
url                27851
closed             27851
review_count       27851
rating             27851
price              23729
lat                27850
long               27850
add1               27496
city               27851
state              27851
zip_code           27822
country            27851
disp_ph            27251
price_attribute    23729
dtype: int64

In [7]:
yelp_format_rd1.dtypes

search_city         object
search_cuisine      object
y_id                object
name                object
image_url           object
url                 object
closed                bool
review_count         int64
rating             float64
price               object
lat                float64
long               float64
add1                object
city                object
state               object
zip_code            object
country             object
disp_ph             object
price_attribute     object
dtype: object

In [8]:
yelp_gb_cities = yelp_format_rd1.groupby(['search_city'])
yelp_gb_cities.sum().head()

,closed,review_count,rating,lat,long
search_city,,,,,
"Albany, New York",False,61218,2506.0,27242.467834,-47077.268325
"Annapolis, Maryland",False,96385,1912.0,19436.953223,-38130.413818
"Atlanta, Georgia",False,287240,3898.0,32554.059412,-81150.804242
"Augusta, Maine",False,15429,1021.0,11696.821371,-18341.337151
"Austin, Texas",False,299061,3792.0,27810.980645,-89635.319822


In [9]:
yelp_gb_cuisines = yelp_format_rd1.groupby(['search_cuisine'])
yelp_gb_cuisines.sum().head()

,closed,review_count,rating,lat,long
search_cuisine,,,,,
American,0.0,658067,9574.5,93656.108174,-222832.643590
Cajun,0.0,364110,5698.0,56090.625394,-139019.652525
Caribbean,0.0,89119,3081.5,29446.371743,-67662.051855
Chinese,0.0,215002,7071.0,73497.901854,-178476.610595
French,0.0,251601,4517.5,43102.898712,-105380.021478


In [10]:
yelp_gb_price = yelp_format_rd1.groupby(['price'])
yelp_gb_price.sum().head()

,closed,review_count,rating,lat,long
price,,,,,
$,2.0,736368,26193.5,257384.932815,-6.172942e+05
$$,0.0,3755842,62684.0,611843.526283,-1.491098e+06
$$$,0.0,334326,4027.0,38330.086618,-9.183231e+04
$$$$,0.0,30459,460.0,3951.409992,-1.070779e+04


In [11]:
yelp_gb_citiescount = yelp_format_rd1.groupby(['search_city'])
yelp_gb_citiescount.count().head()

,search_cuisine,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph,price_attribute
search_city,,,,,,,,,,,,,,,,,,
"Albany, New York",638,638,638,628,638,638,638,638,531,638,638,632,638,638,638,638,622,531
"Annapolis, Maryland",498,498,498,496,498,498,498,498,437,498,498,490,498,498,498,498,493,437
"Atlanta, Georgia",962,962,962,960,962,962,962,962,851,962,962,960,962,962,962,962,947,851
"Augusta, Maine",264,264,264,253,264,264,264,264,210,264,264,264,264,264,264,264,261,210
"Austin, Texas",917,917,917,915,917,917,917,917,768,917,917,910,917,917,916,917,890,768


In [12]:
yelp_gb_cuisinescount = yelp_format_rd1.groupby(['search_cuisine'])
yelp_gb_cuisinescount.count().head()

,search_city,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph,price_attribute
search_cuisine,,,,,,,,,,,,,,,,,,
American,2396,2396,2396,2368,2396,2396,2396,2396,2039,2396,2396,2392,2396,2396,2396,2396,2346,2039
Cajun,1491,1491,1491,1479,1491,1491,1491,1491,1281,1491,1491,1470,1491,1491,1491,1491,1451,1281
Caribbean,764,764,764,738,764,764,764,764,532,764,764,724,764,764,761,764,735,532
Chinese,1914,1914,1914,1887,1914,1914,1914,1914,1716,1914,1914,1912,1914,1914,1913,1914,1883,1716
French,1118,1118,1118,1111,1118,1118,1118,1118,1018,1118,1118,1111,1118,1118,1117,1118,1097,1018
